# Introduction to Spark

In this notebook, you will be introduced to the Apache Spark libary for big data processing. In order to use this notebook, you must openit with `pyspark` rather than `ipython` or `jupyter`. `pyspark` will automatically load some of the Spark internals for you. In particular, the variable `sc` is a Spark context. Check that it is correctly initialised:

In [2]:
sc = SparkContext.getOrCreate()

print sc
print "Ready to go!"

Ready to go!


### Learning activity: Create RDDs

To analyse large datasets using Spark you will load them into Resilient Distributed Datasets (RDDs). There are a number of ways in which you can create RDDs. Use the `parallelize()` function to create one from a Python collection, and use the `textFile()` function to create an RDD from the file `war-and-peace.txt`. 

In [3]:
coll_rdd = sc.parallelize([1,2,3])

In [4]:
wap_rdd = sc.textFile('war-and-peace.txt')
wap_rdd

war-and-peace.txt MapPartitionsRDD[2] at textFile at <unknown>:0

### Learning activity: Basic RDD manipulation

Print the number of lines in War and Peace, and the first 15 lines using the functions `count()` and `take()`.

In [5]:
wap_rdd

war-and-peace.txt MapPartitionsRDD[2] at textFile at <unknown>:0

In [6]:
wap_rdd.take(15)

[u'                                      1869',
 u'                                 WAR AND PEACE',
 u'                                 by Leo Tolstoy',
 u'BK1',
 u'                                 BOOK ONE: 1805',
 u'BK1|CH1',
 u'  CHAPTER I',
 u'',
 u'  "Well, Prince, so Genoa and Lucca are now just family estates of the',
 u"Buonapartes. But I warn you, if you don't tell me that this means war,",
 u'if you still try to defend the infamies and horrors perpetrated by',
 u'that Antichrist- I really believe he is Antichrist- I will have',
 u'nothing more to do with you and you are no longer my friend, no longer',
 u"my 'faithful slave,' as you call yourself! But how do you do? I see",
 u'I have frightened you- sit down and tell me all the news."']

In [7]:
for l in wap_rdd.take(15):
    print(l)

                                      1869
                                 WAR AND PEACE
                                 by Leo Tolstoy
BK1
                                 BOOK ONE: 1805
BK1|CH1
  CHAPTER I

  "Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news."


### Learning activity: `filter()` and `map()` and `distinct()`

Lets apply some transformations onto RDDs. The following helper function will be useful to select the words from a line.

In [8]:
# A helper function to compute the list of words in a line of text
import re
def get_words(line):
    return re.compile('\w+').findall(line)

print get_words("This, is a test!")

['This', 'is', 'a', 'test']


Use `filter()` to count the number of lines which mention `war` and the number of lines which mention `peace`.

In [9]:
# How often are war and peace mentioned?
war_rdd = wap_rdd.filter(lambda line: 'war' in get_words(line))
peace_rdd = wap_rdd.filter(lambda line: 'peace' in get_words(line))

print(war_rdd.count())
print(peace_rdd.count())

265
104


Use `map()` to capitalise each line in the RDD, and print the first 15 capitalized lines.

In [10]:
# Capitalize each line in the RDD
wap_rdd.map(lambda line: line.capitalize()).take(15)

[u'                                      1869',
 u'                                 war and peace',
 u'                                 by leo tolstoy',
 u'Bk1',
 u'                                 book one: 1805',
 u'Bk1|ch1',
 u'  chapter i',
 u'',
 u'  "well, prince, so genoa and lucca are now just family estates of the',
 u"Buonapartes. but i warn you, if you don't tell me that this means war,",
 u'If you still try to defend the infamies and horrors perpetrated by',
 u'That antichrist- i really believe he is antichrist- i will have',
 u'Nothing more to do with you and you are no longer my friend, no longer',
 u"My 'faithful slave,' as you call yourself! but how do you do? i see",
 u'I have frightened you- sit down and tell me all the news."']

Use `flatMap()` to create an RDD of the words in War and Peace and count the number of words.

In [11]:
# Split each line into words using get_words()
wap_words_rdd = wap_rdd.flatMap(lambda line: get_words(line))
wap_words_rdd.count()

573322

Finally, use `distinct()` to count the number of different words in the RDD.

In [12]:
# Count the number of distinct words
wap_words_rdd.distinct().count()

19206

### Learning activity: Set like transformations

Use the functions `union()` and `intersection()` to create RDDs of lines with either war or peace mentioned, and both war and peace being mentioned. Count how many lines of each type there are and print some examples.

In [13]:
war_u_peace = war_rdd.union(peace_rdd)
war_i_peace = war_rdd.intersection(peace_rdd)

print war_u_peace.count()
print war_i_peace.count()

369
7


### Learning activity: `reduce()`

You have already seen three actions: `collect()` which returns all elements in the RDD, `take(n)`, which return the first `n` elements of the RDD, and `count()` which returns the number of elements in the RDD.

The action `reduce()` takes as input a function which collapses two elements into one. Use it to find the longest word in War and Peace.

In [14]:
wap_words_rdd.distinct().reduce(lambda x, y: x if len(x) > len(y) else y)

u'characteristically'

### Bonus activity: merging filters

Find all the lines that mention both war and peace **without** using `intersection()`.

In [15]:
wap_rdd.filter(lambda line: 'war' in get_words(line)).filter(lambda line: 'peace' in get_words(line))

PythonRDD[31] at RDD at PythonRDD.scala:48

### Bonus activity: Finding proper nouns

The Python function `str.istitle()` returns `True` if the string `str` is titlecased: the first character is uppercase and others are lowercase. Use it to:
* Find the set of distinct words in War and Peace which are titlecased
* Find the set of distinct words in War and Peace which are not titlecased

In [16]:
istitle_word = wap_words_rdd.distinct().filter(lambda x: x.istitle())
isntitle_word = wap_words_rdd.distinct().filter(lambda x: not(x.istitle()))

The Python function `str.lower` returns a string with all characters of `str` lowercase. Use it, along with your previously generated RDD to find the set of words in War and Peace which only appear titlecased.

In [17]:
x = istitle_word.map(lambda x: x.lower())
y = isntitle_word.map(lambda x: x.lower())

only_istitle_word = x.subtract(y)
print(only_istitle_word.count())
print(only_istitle_word.take(10))

1541
[u'bennigsens', u'gwiska', u'melyukova', u'hosjeradek', u'femgalka', u'jacob', u'karagina', u'maksim', u'nostitz', u'bolotnoe']


### Learning activity: Partitions

Load War and peace using a minimum of four partitions and use `getNumPartitions()` to show the resulting number used.

In [18]:
print wap_rdd.getNumPartitions()

wap_rdd_4parti = sc.textFile('war-and-peace.txt', minPartitions=4)
print wap_rdd_4parti.getNumPartitions()

2
4


# Key/Value pairs in Spark

### Learning activity: WordCount in Spark

Use the functions `flatMap()` and `reduceByKey()` to count the number of occurences of each word in War and Peace, and print the count of five words.

In [19]:
res = wap_words_rdd.map(lambda x: (x, 1))
res = res.reduceByKey(lambda x, y: x + y)
res.take(5)

[(u'Saints', 1),
 (u'nunnery', 3),
 (u'worshiped', 1),
 (u'yellow', 20),
 (u'four', 98)]

### Learning activity: using `groupByKey()`

Reimplement the above word count using `groupByKey()` instead of `reduceByKey()`

In [20]:
res = wap_words_rdd.map(lambda x: (x, 1))
res = res.groupByKey().mapValues(len)
res.take(5)

[(u'Saints', 1),
 (u'nunnery', 3),
 (u'worshiped', 1),
 (u'yellow', 20),
 (u'four', 98)]

### Learning activity: computing the average of each key

The pair RDD defined below `word_line_pairs` has an element for each line in War and Peace with as key the first word, and as value the line itself. Use it to compute the average length of each line for each starting word.

In [28]:
word_line_pairs = wap_rdd.map(lambda x: (x.strip().split(' ', 1)[0], x))
word_line_pairs.take(10)

def avg_len(x):
    leng = 0
    for i in x:
        leng += len(i)
    return leng

avg_start_word_len = word_line_pairs.groupByKey() \
                                    .mapValues(avg_len)
    
avg_start_word_len.take(3)

[(u'', 0), (u'shouted,', 304), (u'shouted.', 8)]

# Spark in Practice

### Learning activity: Use an accumulator
Reimplement the definition of the `word` (re-expressed below) RDD to count blank lines as you process it.

In [22]:
# Old definition of words
lines = sc.textFile('war-and-peace.txt')
words = lines.flatMap(lambda line: get_words(line))

blank_lines = sc.accumulator(0)

def get_words_count_blank(line):
    global blank_lines
    if line == '':
        blank_lines += 1
    return get_words(line)

words = lines.flatMap(get_words_count_blank)

# Important to run count() before printing blank lines count
# Only actions would activate the accumulator, not transforms
print words.count()
print "Blank lines count: " + str(blank_lines.value)

573322
Blank lines count: 761


### Learning activity: Implement the same program with an action

In [23]:
lines.filter(lambda x: x == '').count()

761

### Learning activity: compare the amount of data shuffled with SparkUI
Using Spark UI, compare the amount of data shuffled by each of the following two (equivalent) computations:

In [24]:
distinct_words = words.distinct()
short_distinct_words = distinct_words.filter(lambda x: len(x)<6)
short_distinct_words.count()

4536

In [25]:
short_words = words.filter(lambda x: len(x)<6)
short_distinct_words = short_words.distinct()
short_distinct_words.count()

4536

Similarly, re-execute the two versions of word count you implemented before and compare the amount of data shuffled using `reduceByKey()` and using `groupByKey()`.

In [26]:
warLines = lines.filter(lambda line: "war" in get_words(line))
peaceLines = lines.filter(lambda line: "peace" in get_words(line))

warAndPeaceLines = warLines.intersection(peaceLines)
warOrPeaceLines = warLines.union(peaceLines)

print "There are %d lines with either war or peace, some of them are:" % warOrPeaceLines.count()
for line in warOrPeaceLines.take(5):
    print line
    
print "------------------------------------------"

print "There are %d lines with both war and peace, some of them are:" % warAndPeaceLines.count()
for line in warAndPeaceLines.take(5):
    print line

There are 369 lines with either war or peace, some of them are:
Buonapartes. But I warn you, if you don't tell me that this means war,
things, but Austria never has wished, and does not wish, for war.
to get himself killed. Tell me what this wretched war is for?" she
  "You are off to the war, Prince?" said Anna Pavlovna.
tell you. There is a war now against Napoleon. If it were a war for
------------------------------------------
There are 7 lines with both war and peace, some of them are:
blamed," he said, "both for that war and the peace... but everything
peace nor war, neither an advance nor a defensive camp at the Drissa
perpetual peace and the abolition of war, and secondly, by the fact
war and the peace that had been concluded. "Yes, I have been much
  "To enter Russia without declaring war! I will not make peace as
